# Problem Statement

here

In [55]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

import plotly.express as px
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder, LabelEncoder

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR, LinearSVR

import lightgbm as lgb
import shap

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("./data/year_2016_clean.csv", sep=";")
df.head()

,OSEBuildingID,BuildingType,PrimaryPropertyType,CouncilDistrictCode,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,ComplianceStatus,Outlier,ZipCode,Latitude,Longitude,NbProperty,BuildingAge
0,1,NonResidential,hotel_catering -,7,downtown,1,12,88434,0.0,88434.0,hotel_catering -,hotel_catering -,88434.0,none,0.0,none,0.0,60.0,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2003882.00,1.156514e+06,3946027.0,12764.52930,1276453.0,249.98,2.83,Compliant,NaN,98101,47.61220,-122.33799,1,89
1,2,NonResidential,hotel_catering -,7,downtown,1,11,103566,15064.0,88502.0,hotel_catering -,hotel_catering -,83880.0,none,15064.0,hotel_catering -,4622.0,61.0,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.00,9.504252e+05,3242851.0,51450.81641,5145082.0,295.86,2.86,Compliant,NaN,98101,47.61317,-122.33393,3,20
2,3,NonResidential,hotel_catering -,7,downtown,1,41,956110,196718.0,759392.0,hotel_catering -,hotel_catering -,756493.0,none,0.0,none,0.0,43.0,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,21566554.00,1.451544e+07,49526664.0,14938.00000,1493800.0,2089.28,2.19,Compliant,NaN,98101,47.61393,-122.33810,1,47
3,5,NonResidential,hotel_catering -,7,downtown,1,10,61320,0.0,61320.0,hotel_catering -,hotel_catering -,61320.0,none,0.0,none,0.0,56.0,110.800003,113.300003,216.199997,224.000000,6794584.0,6946800.5,2214446.25,8.115253e+05,2768924.0,18112.13086,1811213.0,286.43,4.67,Compliant,NaN,98101,47.61412,-122.33664,1,90
4,8,NonResidential,hotel_catering -,7,downtown,1,18,175580,62000.0,113580.0,hotel_catering - leisure -,hotel_catering -,123445.0,none,68009.0,leisure -,0.0,75.0,114.800003,118.699997,211.399994,215.600006,14172606.0,14656503.0,0.00,1.573449e+06,5368607.0,88039.98438,8803998.0,505.01,2.88,Compliant,NaN,98121,47.61375,-122.34047,3,36


---

# Feature Engineering

In [30]:
df["RateParking"] = round(df.PropertyGFAParking / df.PropertyGFATotal, 4)
df["RateBuilding"] = round(df['PropertyGFABuilding(s)'] / df.PropertyGFATotal, 4)
df["RatePerFloors"] = round(df.PropertyGFATotal / df.NumberofFloors, 4)
df["RatePerBuildings"] = round(df.PropertyGFATotal / df.NumberofBuildings, 4)
df["RateLargestPropertyUseType"] = round(df.LargestPropertyUseTypeGFA / df.PropertyGFATotal, 4)
df["RateSecondLargestPropertyUseType"] = round(df.SecondLargestPropertyUseTypeGFA / df.PropertyGFATotal, 4)
df["RateThirstLargestPropertyUseType"] = round(df.ThirdLargestPropertyUseTypeGFA / df.PropertyGFATotal, 4)
df["NumberOfAllUseTypes"] = df.ListOfAllPropertyUseTypes.str.count(" - ") + 1

useless 'CouncilDistrictCode', 'ListOfAllPropertyUseTypes', 'ENERGYSTARScore'

target 'TotalGHGEmissions'  
supprimer 'GHGEmissionsIntensity'

In [31]:
data = df.drop(columns=['CouncilDistrictCode', 'ListOfAllPropertyUseTypes', 'ENERGYSTARScore', 'GHGEmissionsIntensity', 'Outlier'])
print(f"df shape: {data.shape}")


# let's shuffle our dataset
for i in range(10):
    data = data.sample(frac=1)
    # from sklearn.utils import shuffle
    # data = shuffle(data)


X = data.drop(columns = ['TotalGHGEmissions'])
y = data[['TotalGHGEmissions']]

assert X.isnull().sum().sum() == 0
assert y.isnull().sum().sum() == 0

df shape: (3353, 41)


shuffle + dummy regressor

In [32]:
# general seed for sk learn
np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Scaling

In [33]:
num_cols = X.select_dtypes(include='number').columns
print(f"nb of numerical cols that are scaled: {len(num_cols)}")

robust_scaler = RobustScaler()

X_train_num_sc = pd.DataFrame(
    robust_scaler.fit_transform(X_train[num_cols]),
    columns=num_cols,
    index=X_train.index
)

X_test_num_sc = pd.DataFrame(
    robust_scaler.transform(X_test[num_cols]),
    columns=num_cols,
    index=X_test.index
)

assert X_train_num_sc.shape == (X_train.shape[0], len(num_cols)) 
assert X_test_num_sc.shape == (X_test.shape[0], len(num_cols)) 

nb of numerical cols that are scaled: 33


OHE

In [34]:
# df.isin([np.inf, -np.inf]).sum()


cat_cols = X_train.select_dtypes(exclude='number').columns
print(f"nb of categorical cols that are scaled: {len(num_cols)}")

ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

X_train_cat_enc = ohe.fit_transform(X_train[cat_cols])
X_train_cat_enc = pd.DataFrame(
    X_train_cat_enc,
    columns=ohe.get_feature_names_out(X_train.select_dtypes(exclude='number').columns),
    index=X_train.index
)

X_test_cat_enc = ohe.transform(X_test[cat_cols])
X_test_cat_enc = pd.DataFrame(
    X_test_cat_enc,
    columns=ohe.get_feature_names_out(X_test.select_dtypes(exclude='number').columns),
    index=X_test.index
)


assert X_train_cat_enc.shape[0] == X_train.shape[0] 
assert X_test_cat_enc.shape[0] == X_test.shape[0]

nb of categorical cols that are scaled: 33


Concatenation

In [35]:
assert all(df.isnull().sum().sum() == 0 for df in [X_train_cat_enc, X_test_cat_enc, X_train_num_sc, X_test_num_sc])

X_train_enc = pd.concat([X_train_num_sc, X_train_cat_enc], axis=1)
X_test_enc = pd.concat([X_test_num_sc, X_test_cat_enc], axis=1)

X_train_enc.shape, X_test_enc.shape

# X_test_enc[[col for col in X_test_enc.columns if X_test_enc[col].hasnans]].isnull().sum()

((2682, 118), (671, 118))

Modeling

In [36]:
def regression(model, name, X, y):
    metrics = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
    score = cross_validate(model, X, y , cv=5, scoring=metrics, return_train_score=True)
    
    dic = {
        'model ': [name],
        'train r²': [score['train_r2'].mean()],
        'test r²': [score['test_r2'].mean()],
        'train RMSE': [np.sqrt(- (score['train_neg_mean_squared_error'].mean()))],
        'test RMSE': [np.sqrt(- (score['test_neg_mean_squared_error'].mean()))],
        'train MAE': [np.sqrt(- (score['train_neg_mean_absolute_error'].mean()))],
        'test MAE': [np.sqrt(- (score['test_neg_mean_absolute_error'].mean()))],
    }
    return pd.DataFrame(dic)

In [37]:
def compare_results(X, y):
    results = pd.DataFrame()
    model = []
    model.append(('dummy_regressor', DummyRegressor()))
    model.append(('svr', SVR()))
    model.append(('decision_tree', DecisionTreeRegressor()))
    model.append(('random_forest', RandomForestRegressor()))
    model.append(('adaBoost', AdaBoostRegressor()))
    model.append(('lgbm', lgb.LGBMRegressor()))
    
    for name, mod in model:
        res = regression(mod, name, X, y)
        results = pd.concat([results, res], ignore_index=True)
    return results.style.hide_index()

In [38]:
compare_results(X_train_enc, y_train)

model,train r²,test r²,train RMSE,test RMSE,train MAE,test MAE
dummy_regressor,0.000000,-0.012581,552.748411,553.353064,11.952210,11.958761
svr,-0.019482,-0.038691,557.994042,558.135052,10.412142,10.415952
decision_tree,1.000000,0.413254,0.000000,326.093212,0.000000,5.204489
random_forest,0.950153,0.740359,122.707737,310.079129,2.897012,4.751326
adaBoost,0.913032,0.207678,143.745576,416.418012,10.612703,11.567421
lgbm,0.793153,0.425896,250.092542,387.610370,5.581483,6.919425


In [42]:
model_list = [
    DummyRegressor(),
    SVR(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    AdaBoostRegressor(),
    lgb.LGBMRegressor(),
    GradientBoostingRegressor(),
    ExtraTreesRegressor()
]

In [48]:
model_names = [str(m)[:str(m).index('(')] for m in model_list]
rmse_train, rmse_test = [], []

def get_rmse(reg, model_name):
    """Print the score for the model passed in argument & return scores for the train / test sets"""
    y_train_pred, y_pred = reg.predict(X_train_enc), reg.predict(X_test_enc)
    rmse_train, rmse_test = np.sqrt(mean_squared_error(y_train, y_train_pred)), np.sqrt(mean_squared_error(y_test, y_pred))
    # print(model_name, f'\t - RMSE on training = {rmse_train:.2f} / on test = {rmse_test:.2f}')
    return rmse_train, rmse_test


# fit and predict all models
for model, name in zip(model_list, model_names):
    model.fit(X_train_enc, y_train)
    sc_train, sc_test = get_rmse(model, name)
    rmse_train.append(sc_train)
    rmse_test.append(sc_test)
    

df_score = pd.DataFrame({
    'model_names': model_names,
    'rmse_train': rmse_train,
    'rmse_test': rmse_test
})

df_score.round(1)

,model_names,rmse_train,rmse_test
0,DummyRegressor,552.8,484.5
1,SVR,557.6,487.4
2,DecisionTreeRegressor,0.0,83.9
3,RandomForestRegressor,124.1,68.6
4,AdaBoostRegressor,101.2,122.7
5,LGBMRegressor,226.0,186.2
6,GradientBoostingRegressor,7.1,62.0
7,ExtraTreesRegressor,0.0,27.8


In [52]:
df_score_melt = pd.melt(df_score, id_vars=['model_names'], value_vars=['rmse_train', 'rmse_test'])


px.histogram(
    df_score_melt,
    x="model_names",
    y="value",
    color='variable',
    barmode='group',
    width=800,
    height=500
).show()


# same thing with seaborn
# plt.figure(figsize=(12, 10))
# sns.barplot(y="model_names", x="value", hue="variable", data=df_score_melt)

Side note: here except the dummy regressor and SVM we are only using tree based models. With only tree based models we could have used label encoding instead of ohe in order to deal with less dimensions and for better performances.

We can see that tree based models are performing better. So for the 2nd round we'll keep only them!

In [60]:
for col in cat_cols:
    le = LabelEncoder().fit(X_train[col])
    X_train[col], X_test[col] = le.transform(X_train[col]), le.transform(X_test[col])

In [53]:
lgb.__version__

'3.2.1'

In [ ]:
https://www.kaggle.com/code/hmendonca/lightgbm-predictions-explained-with-shap-0-796

In [ ]:
shap_values = shap.TreeExplainer(lgbm).shap_values(valid_x)

In [ ]:
shap.summary_plot(shap_values, valid_x)


In [ ]:
shap.dependence_plot("EXT_SOURCE_2", shap_values, valid_x)
